In [60]:
import pandas as pd
import numpy as np
import os

from sklearn.impute import KNNImputer

from imblearn.over_sampling import SMOTE

In [3]:
static = pd.read_csv('/Users/von/PycharmProjects/SPH6004_A2/data/static.csv')
dynamic = pd.read_csv('/Users/von/PycharmProjects/SPH6004_A2/data/dynamic.csv')

## Data Description

The static file is on subject level, while the dynamic file is not (in other words, in static, one person per row; in dynamic, several rows per person). 
Then two ideas to deal with the dynamic file:
1) group by id, and take the mean of each lab test;
2) pivot long to wide.

In [8]:
static.shape

(20414, 25)

In [9]:
dynamic.shape

(93604, 70)

In [11]:
## take mean of each lab test
dynamic1 = dynamic.groupby('id').mean(numeric_only=True).reset_index()

In [13]:
df = pd.merge(static, dynamic1, on='id', how='left')

In [14]:
df.shape

(20414, 93)

In [15]:
df.columns

Index(['id', 'hosp_admittime', 'hosp_dischtime', 'icu_intime', 'icu_outtime',
       'los_icu', 'icu_death', 'gender', 'race', 'admission_age',
       'weight_admit', 'height', 'admission_type', 'first_careunit',
       'charlson_score', 'atrial_fibrillation', 'malignant_cancer', 'chf',
       'ckd', 'cld', 'copd', 'diabetes', 'hypertension', 'ihd', 'stroke',
       'albumin', 'globulin', 'total_protein', 'aniongap', 'bicarbonate',
       'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium',
       'potassium', 'heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate',
       'temperature', 'spo2', 'hematocrit', 'hemoglobin', 'mch', 'mchc', 'mcv',
       'platelet', 'rbc', 'rdw', 'wbc', 'alt', 'ast', 'alp', 'amylase',
       'bilirubin_total', 'bilirubin_direct', 'bilirubin_indirect', 'ck_cpk',
       'ck_mb', 'ggt', 'ld_ldh', 'so2', 'po2', 'pco2', 'fio2_chartevents',
       'aado2', 'aado2_calc', 'pao2fio2ratio', 'ph', 'baseexcess',
       'bicarbonate_bg', 'totalco2', 'hematocrit_bg'

In [17]:
df.dtypes

id                  int64
hosp_admittime     object
hosp_dischtime     object
icu_intime         object
icu_outtime        object
                   ...   
thrombin          float64
inr               float64
pt                float64
ptt               float64
urineoutput       float64
Length: 93, dtype: object

## Check Missing

Some of the variables have large percentage of missing. Here remove columns with > 90% missing. Set this bar because the dbp, sbp and mbp have 85% missing, and those may have important clinical meaning and prefer to remain them.

In [16]:
na_df = df.isnull().sum()*100/len(df)
nadf_reset = na_df.reset_index()
nadf_reset.columns = ['Row Name', 'Missing Percentage']
nadf_reset

,Row Name,Missing Percentage
0,id,0.000000
1,hosp_admittime,0.000000
2,hosp_dischtime,0.000000
3,icu_intime,0.000000
4,icu_outtime,0.000000
...,...,...
88,thrombin,99.823650
89,inr,12.511022
90,pt,12.511022
91,ptt,13.319291


In [30]:
# remove cols that have more than 90% missing
df1 = df.loc[:, df.columns[nadf_reset['Missing Percentage'] < 90]]

## Dummy Variable

`gender`, `race`, `admission_type`, and `first_careunit` are changed to dummy.
The `race` includes many categories. Now recategorized into 4 major ethnic groups. For UNKNOWN or UNABLE TO OBTAIN or PATIENT DECLINED TO ANSWER, they are defined as the largest ethnic group WHITE.
The `admission_type` is recategorized into 3 major admission types.


In [31]:
## gender
gender_df_wide = pd.get_dummies(df1['gender'], prefix = 'gender', dtype = int)
df1 = df1.drop('gender', axis=1)
df1 = pd.concat((df1, gender_df_wide), axis = 1)

In [23]:
## race
df1['race'].value_counts()

WHITE                                        13133
BLACK/AFRICAN AMERICAN                        2332
UNKNOWN                                       1605
OTHER                                          606
WHITE - OTHER EUROPEAN                         350
HISPANIC/LATINO - PUERTO RICAN                 274
WHITE - RUSSIAN                                247
ASIAN                                          197
UNABLE TO OBTAIN                               180
ASIAN - CHINESE                                167
HISPANIC OR LATINO                             164
HISPANIC/LATINO - DOMINICAN                    142
BLACK/CAPE VERDEAN                             139
BLACK/CARIBBEAN ISLAND                         110
ASIAN - SOUTH EAST ASIAN                        93
PATIENT DECLINED TO ANSWER                      91
BLACK/AFRICAN                                   89
PORTUGUESE                                      86
HISPANIC/LATINO - GUATEMALAN                    50
AMERICAN INDIAN/ALASKA NATIVE  

In [32]:
conditions = [
    (df1['race'].str.contains('WHITE', case=False)),
    (df1['race'].str.contains('BLACK', case=False)),
    (df1['race'].str.contains('HISPANIC', case=False)),
    (df1['race'].str.contains('ASIAN', case=False)),
    (df1['race'].str.contains('UNKNOWN|UNABLE|DECLINED', case=False)),
    (df1['race'].str.contains('OTHER|PORTUGUESE|ALASKA|HAWAIIAN|MULTIPLE|SOUTH AMERICAN', case=False))
]
values = ['WHITE', 'BLACK', 'HISPANIC/LATINO', 'ASIAN', 'WHITE', 'OTHER']

df1['race'] = np.select(conditions, values)

In [33]:
df1['race'].value_counts()

WHITE              15670
BLACK               2670
OTHER                813
HISPANIC/LATINO      747
ASIAN                514
Name: race, dtype: int64

In [34]:
race_df_wide = pd.get_dummies(df1['race'], prefix = 'race', dtype = int)
df1 = df1.drop('race', axis=1)
df1 = pd.concat((df1, race_df_wide), axis = 1)

In [36]:
## admission type
df['admission_type'].value_counts()

EW EMER.                       11400
URGENT                          3630
OBSERVATION ADMIT               2486
SURGICAL SAME DAY ADMISSION     1411
DIRECT EMER.                     878
ELECTIVE                         592
EU OBSERVATION                    13
DIRECT OBSERVATION                 4
Name: admission_type, dtype: int64

In [37]:
conditions = [
    (df1['admission_type'].str.contains('EMER', case=False)),
    (df1['admission_type'].str.contains('URGENT|SURGICAL', case=False)),
    (df1['admission_type'].str.contains('ELECTIVE|OBSERVATION', case=False))
]
values = ['EMERGENCY', 'URGENT', 'ELECTIVE']

df1['admission_type'] = np.select(conditions, values)

In [39]:
adm_df_wide = pd.get_dummies(df1['admission_type'], prefix = 'admtyp', dtype = int)
df1 = df1.drop('admission_type', axis=1)
df1 = pd.concat((df1, adm_df_wide), axis = 1)

In [40]:
## first care unit
df1['first_careunit'].value_counts()

Medical Intensive Care Unit (MICU)                  5374
Medical/Surgical Intensive Care Unit (MICU/SICU)    3952
Surgical Intensive Care Unit (SICU)                 3100
Coronary Care Unit (CCU)                            2475
Trauma SICU (TSICU)                                 2452
Cardiac Vascular Intensive Care Unit (CVICU)        2422
Neuro Surgical Intensive Care Unit (Neuro SICU)      435
Neuro Intermediate                                   145
Neuro Stepdown                                        59
Name: first_careunit, dtype: int64

In [41]:
conditions = [
    (df1['first_careunit'].str.contains('MEDICAL|SURGICAL', case=False)),
    (df1['first_careunit'].str.contains('NEURO', case=False)),
    (df1['first_careunit'].str.contains('CORONARY', case=False)),
    (df1['first_careunit'].str.contains('TRAUMA', case=False)),
    (df1['first_careunit'].str.contains('CARDIAC', case=False))
]
values = ['MEDICAL/SURGICAL', 'NEURO', 'CORONARY', 'TRAUMA', 'CARDIAC']

df1['first_careunit'] = np.select(conditions, values)

In [42]:
fc_df_wide = pd.get_dummies(df1['first_careunit'], prefix = 'firstcare', dtype = int)
df1 = df1.drop('first_careunit', axis=1)
df1 = pd.concat((df1, fc_df_wide), axis = 1)

## Normalization

In [55]:
df2 = df1

In [56]:
## Normalization
def normalize_column(x):
        return (x - x.mean()) / x.std()

cols_norm1 = ['admission_age', 'weight_admit', 'height', 'charlson_score']

df2[cols_norm1] = df2[cols_norm1].apply(normalize_column)
df2.loc[:, 'albumin':'ptt'] = df2.loc[:, 'albumin':'ptt'].apply(normalize_column)

In [48]:
df2

,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,admission_age,weight_admit,height,...,race_OTHER,race_WHITE,admtyp_ELECTIVE,admtyp_EMERGENCY,admtyp_URGENT,firstcare_CARDIAC,firstcare_CORONARY,firstcare_MEDICAL/SURGICAL,firstcare_NEURO,firstcare_TRAUMA
0,24099382,6/11/11 7:15,6/18/11 21:45,6/11/11 14:31,6/13/11 18:05,2.15,0,-0.560507,0.230368,0.051240,...,1,0,1,0,0,0,0,0,0,1
1,27824879,9/30/28 20:55,10/10/28 15:54,9/30/28 20:58,10/5/28 18:39,4.90,0,0.806297,-1.076268,NaN,...,0,1,0,1,0,0,0,0,0,1
2,27146619,9/20/41 0:34,9/22/41 12:05,9/20/41 7:13,9/21/41 18:10,1.46,0,0.932592,-0.935010,NaN,...,0,1,0,1,0,0,0,1,0,0
3,22476441,10/8/73 16:46,10/16/73 13:05,10/8/73 18:28,10/10/73 13:26,1.79,0,-0.218748,-0.419419,-0.873545,...,0,0,0,1,0,0,0,1,0,0
4,23169158,3/12/27 1:19,3/19/27 13:15,3/12/27 2:43,3/15/27 1:53,2.97,0,1.091332,-0.225189,-0.873545,...,0,1,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20409,29968077,9/12/59 6:20,10/5/59 15:41,9/12/59 7:45,9/18/59 16:55,6.38,0,0.418001,1.296866,0.051240,...,0,1,0,1,0,0,1,0,0,0
20410,29975784,8/30/36 15:18,9/2/36 16:15,8/30/36 16:48,9/1/36 10:41,1.75,0,1.249892,-0.571271,NaN,...,0,1,0,1,0,0,0,1,0,0
20411,29987115,2/16/48 17:42,2/20/48 19:00,2/16/48 17:43,2/20/48 21:00,4.14,1,-1.542852,-0.652494,-1.613373,...,0,1,0,0,1,0,0,1,0,0
20412,29990494,5/10/71 12:05,6/16/71 13:45,5/17/71 22:16,5/28/71 20:41,10.93,0,-0.630271,0.576450,-0.133717,...,0,1,0,1,0,0,0,1,0,0


## KNN Imputation

In [51]:
imputer = KNNImputer(n_neighbors=3, weights='uniform', metric='nan_euclidean')
df_impute = df2.loc[:, 'admission_age':]
df_impute1 = imputer.fit_transform(df_impute)

In [57]:
df2 = df2.drop(df_impute.columns, axis=1)
df_impute1 = pd.DataFrame(df_impute1)
df_impute1.columns = df_impute.columns
df2 = pd.concat((df2, df_impute1), axis=1)

In [58]:
df2

,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,admission_age,weight_admit,height,...,race_OTHER,race_WHITE,admtyp_ELECTIVE,admtyp_EMERGENCY,admtyp_URGENT,firstcare_CARDIAC,firstcare_CORONARY,firstcare_MEDICAL/SURGICAL,firstcare_NEURO,firstcare_TRAUMA
0,24099382,6/11/11 7:15,6/18/11 21:45,6/11/11 14:31,6/13/11 18:05,2.15,0,-0.560507,0.230368,0.051240,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,27824879,9/30/28 20:55,10/10/28 15:54,9/30/28 20:58,10/5/28 18:39,4.90,0,0.806297,-1.076268,-0.842718,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,27146619,9/20/41 0:34,9/22/41 12:05,9/20/41 7:13,9/21/41 18:10,1.46,0,0.932592,-0.935010,-0.472804,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,22476441,10/8/73 16:46,10/16/73 13:05,10/8/73 18:28,10/10/73 13:26,1.79,0,-0.218748,-0.419419,-0.873545,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,23169158,3/12/27 1:19,3/19/27 13:15,3/12/27 2:43,3/15/27 1:53,2.97,0,1.091332,-0.225189,-0.873545,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20409,29968077,9/12/59 6:20,10/5/59 15:41,9/12/59 7:45,9/18/59 16:55,6.38,0,0.418001,1.296866,0.051240,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
20410,29975784,8/30/36 15:18,9/2/36 16:15,8/30/36 16:48,9/1/36 10:41,1.75,0,1.249892,-0.571271,2.085767,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
20411,29987115,2/16/48 17:42,2/20/48 19:00,2/16/48 17:43,2/20/48 21:00,4.14,1,-1.542852,-0.652494,-1.613373,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
20412,29990494,5/10/71 12:05,6/16/71 13:45,5/17/71 22:16,5/28/71 20:41,10.93,0,-0.630271,0.576450,-0.133717,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


## Outcome Distribution: Oversampling

In [59]:
df2['icu_death'].value_counts()

0    18276
1     2138
Name: icu_death, dtype: int64

In [62]:
X_df = df2.iloc[:, 7:]
y_df = df2.loc[:, 'icu_death']

In [64]:
# Oversampling
smote_sampler = SMOTE(random_state=12, sampling_strategy='minority')
X_df_SMOTE, y_df_SMOTE = smote_sampler.fit_resample(X_df, y_df)

In [65]:
X_df = pd.DataFrame(X_df_SMOTE)
y_df = pd.DataFrame(y_df_SMOTE)

In [72]:
df3 = df2

In [73]:
df3 = df3.drop(X_df.columns, axis=1)
df3 = df3.drop('icu_death', axis=1)
df3 = pd.concat((df3, y_df), axis=1)
df3 = pd.concat((df3, X_df), axis=1)

In [75]:
df3.shape

(36552, 72)